In [1]:
from simulate_s2s_uns2uns_model import simulate_steps
import analyse_results
import pandas as pd


#Generate metrics/model information for dictionary of networks
#and their detections data
def generate_model_data(ntwkDict):
    #metrics information needed
    model_data = ['P_s2s','P_u2s','P_visible','seen','seen2seen',\
                 'seen2unseen','unseen','unseen2seen','unseen2unseen']
    model_names = list(ntwkDict.keys())
    df = pd.DataFrame(index=model_data,columns=model_names)
    for model in model_names:
        #compute data for each litter
        ntwkDF = ntwkDict[model]
        
        ntwkDF.loc[:,'P_s2s'] = \
        ntwkDF['seen2seen']/(ntwkDF['seen2seen']+ntwkDF['seen2unseen'])
        
        ntwkDF.loc[:,'P_u2s'] = \
        ntwkDF['unseen2seen']/(ntwkDF['unseen2seen']+ntwkDF['unseen2unseen'])
        
        ntwkDF.loc[:,'P_visible'] = ntwkDF['seen']/ntwkDF['visible']

        #summarize data into new df
        for data in model_data:
            if 'P_' in data:
                df.loc[data,model] = f'${ntwkDF[data].mean():.4f} \pm {ntwkDF[data].std():.4f}$'
            else:
                df.loc[data,model] = f'{int(ntwkDF[data].sum())}'
    return df
    

In [2]:
def read_probability_file(fname,fps,nafilter):
    means = pd.read_csv(fname.format(fps,'means',nafilter),
                       header=0,index_col=0)
    stddev = pd.read_csv(fname.format(fps,'stddev',nafilter),
                       header=0,index_col=0)
    ntwkDict = {}
    tex_df = pd.DataFrame()
    for ntwk in means.columns:
        ntwkDict[ntwk] = ((means.loc['P_s2s',ntwk],stddev.loc['P_s2s',ntwk]),\
                          (means.loc['P_u2s',ntwk],stddev.loc['P_u2s',ntwk]))
        tex_df.loc['P_s2s',ntwk] = f"${means.loc['P_s2s',ntwk]:.4f} \pm {stddev.loc['P_s2s',ntwk]:.4f}$"
        tex_df.loc['P_u2s',ntwk] = f"${means.loc['P_u2s',ntwk]:.4f} \pm {stddev.loc['P_u2s',ntwk]:.4f}$"
        tex_df.loc['P_visible',ntwk] = f"${means.loc['P_visible',ntwk]:.4f} \pm {stddev.loc['P_visible',ntwk]:.4f}$"
    return means,stddev,ntwkDict,tex_df

In [3]:
def simulate_and_analyse_ntwk_data(ntwkDict):
    simDict = {}
    for ntwk in ntwkDict:
        print(ntwk)
        p_s2s,p_u2s = ntwkDict[ntwk]
        simDict[ntwk] = simulate_steps(p_s2s,p_u2s,mode,nlitter,tsteps)
    return generate_model_data(simDict)

In [22]:
nlitter = 100
tsteps = 100
fname = 'probabilities/{0}fps/TPandFN_probability_{1}_nafilter_{2}.csv'
mode='mean'
fps=1
nafilter='0'
means,stddev,ntwkDict,tex_df = read_probability_file(fname,fps,nafilter)
sim_data = simulate_and_analyse_ntwk_data(ntwkDict)

mSSD-124
 99                                                                                                                                                                                                                            
mSSD-220
 99                                                                                                                                                                                                                                               
yolov3-tiny-128
 99                                                                                                                                                                                                                           
yolov3-tiny-224
 99                                                                                                                                                                                                                                               


In [23]:
probabilities=['P_u2s','P_s2s','P_visible']
display('experiment data',tex_df.loc[probabilities,:])
display('simulated data',sim_data.loc[probabilities,:])

'experiment data'

,mSSD-124,mSSD-220,yolov3-tiny-128,yolov3-tiny-224
P_u2s,$0.3827\pm0.3434$,$0.4916\pm0.4081$,$0.1310\pm0.2341$,$0.1983\pm0.2976$
P_s2s,$0.4820\pm0.4659$,$0.6352\pm0.4053$,$0.1300\pm0.3170$,$0.2037\pm0.3648$
P_visible,$0.4185\pm0.2923$,$0.6042\pm0.2852$,$0.1214\pm0.2032$,$0.2039\pm0.2712$


'simulated data'

,mSSD-124,mSSD-220,yolov3-tiny-128,yolov3-tiny-224
P_u2s,$0.3811 \pm 0.0676$,$0.4976 \pm 0.0720$,$0.1324 \pm 0.0370$,$0.1924 \pm 0.0476$
P_s2s,$0.4741 \pm 0.0741$,$0.6224 \pm 0.0610$,$0.1340 \pm 0.0855$,$0.1925 \pm 0.0959$
P_visible,$0.4198 \pm 0.0533$,$0.5680 \pm 0.0509$,$0.1323 \pm 0.0351$,$0.1925 \pm 0.0381$


In [18]:
nlitter = 1000
tsteps = 100
mode='mean'
fps=50
nafilter='0'
fname = '../data/simplified_data/first_and_last_appearance_condition/{0}fps/{0}fps-ge_threshold-1/TPandFN_probability_{1}_nafilter_{2}.csv'
means,stddev,ntwkDict,tex_df = read_probability_file(fname,fps,nafilter)
sim = simulate_and_analyse_ntwk_data(ntwkDict)

probabilities=['P_u2s','P_s2s','P_visible']
# display('experiment data',tex_df.loc[probabilities,:])
display('simulated data',sim.loc[probabilities,:])

mSSD-124
 99                                                                                                                                                                                                                                                
mSSD-220
 99                                                                                                                                                                                                                                                                   
yolov3-tiny-128
 99                                                                                                                                                                                                                                                                   
yolov3-tiny-224
 99                                                                                                                                                                                 

'simulated data'

,mSSD-124,mSSD-220,yolov3-tiny-128,yolov3-tiny-224
P_u2s,$0.1351 \pm 0.0468$,$0.2707 \pm 0.0806$,$0.0234 \pm 0.0156$,$0.0570 \pm 0.0246$
P_s2s,$0.7763 \pm 0.0806$,$0.8499 \pm 0.0463$,$0.3285 \pm 0.2509$,$0.4918 \pm 0.1840$
P_visible,$0.3756 \pm 0.1094$,$0.6294 \pm 0.0968$,$0.0383 \pm 0.0289$,$0.1089 \pm 0.0535$


In [40]:
ps2s,pu2s = ntwkDict['yolov3-tiny-128']
simulate_steps(ps2s,pu2s,mode,nlitter,tsteps)

 99                                                                                                                                                                                                                            


,seen2seen,seen2unseen,unseen2seen,unseen2unseen,seen,unseen,visible,0,1,2,...,90,91,92,93,94,95,96,97,98,99
L0,6,17,17,59,23,77,100,0,0,0,...,0,0,0,1,0,0,0,1,0,0
L1,1,8,7,83,9,91,100,1,0,0,...,0,1,0,0,0,0,0,0,0,0
L2,2,15,15,67,17,83,100,0,0,0,...,0,0,1,0,0,1,0,1,0,0
L3,3,9,9,78,12,88,100,0,0,0,...,0,0,0,0,0,0,0,0,0,0
L4,2,9,9,79,11,89,100,0,0,1,...,0,0,0,0,0,0,0,0,0,0
L5,1,14,14,70,15,85,100,0,0,0,...,0,0,0,0,0,0,0,0,0,0
L6,3,16,16,64,19,81,100,0,0,0,...,0,1,0,0,0,1,0,0,0,0
L7,8,19,20,52,28,72,100,0,0,1,...,0,0,0,1,0,1,0,1,1,1
L8,5,19,19,56,24,76,100,0,0,0,...,0,0,0,0,0,0,0,1,0,0
L9,4,12,12,71,16,84,100,0,0,0,...,0,0,1,0,0,0,0,0,0,0
